In [ ]:
import os
import torch
import torchvision
import xml.etree.ElementTree as ET
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as T
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tqdm import tqdm
import gc

class VOCDataset(Dataset):
    def __init__(self, root, image_set="train", transforms=None):
        self.root = root
        self.transforms = transforms
        image_set_file = os.path.join(root, "ImageSets", "Main", f"{image_set}.txt")
        with open(image_set_file) as f:
            self.image_ids = [x.strip() for x in f.readlines()]
        self.img_dir = os.path.join(root, "JPEGImages")
        self.ann_dir = os.path.join(root, "Annotations")
        self.classes = ["__background__"] + self._find_classes()
        self.image_files = sorted([f for f in os.listdir(self.img_dir) if f.endswith((".jpg", ".png"))])

    def _find_classes(self):
        cls = set()
        for fname in os.listdir(self.ann_dir):
            tree = ET.parse(os.path.join(self.ann_dir, fname))
            for obj in tree.findall("object"):
                cls.add(obj.find("name").text)
        return sorted(list(cls))

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        img_path = os.path.join(self.img_dir, f"{image_id}.jpg")
        ann_path = os.path.join(self.ann_dir, f"{image_id}.xml")

        img = Image.open(img_path).convert("RGB")
        tree = ET.parse(ann_path)
        root = tree.getroot()

        boxes, labels = [], []
        for obj in root.findall("object"):
            label = obj.find("name").text
            labels.append(self.classes.index(label))

            bndbox = obj.find("bndbox")
            bbox = [
                float(bndbox.find("xmin").text),
                float(bndbox.find("ymin").text),
                float(bndbox.find("xmax").text),
                float(bndbox.find("ymax").text),
            ]
            boxes.append(bbox)

        if len(boxes) == 0:
            # 🔁 Prova a caricare un'altra immagine se questa è vuota
            return self.__getitem__((idx + 1) % len(self))

            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([idx]),
        }

        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.image_ids)

def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)   
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

def collate_fn(batch):
    return tuple(zip(*batch))


import pandas as pd
from torchmetrics.detection.mean_ap import MeanAveragePrecision

def evaluate_metrics(model, data_loader, device,epoch, csv_path="results/metrics.csv"):
    mAP50 = 0.0
    model.eval()
    metric = MeanAveragePrecision()
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)

    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Evaluating"):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            outputs = model(images)
            metric.update(outputs, targets)

    results = metric.compute()

    # Print
    print("Evaluation Metrics:")
    for k, v in results.items():
        if isinstance(v, torch.Tensor):
            if(k=="map_50"):
                print(f"{k}: {v.item() if v.numel() == 1 else v}")
                mAP50 = v.item()
    metric.reset()
    # Save to CSV
    summary = {k: v.item() if isinstance(v, torch.Tensor) and v.numel() == 1 else str(v)
               for k, v in results.items()}
    #summary["epoch"] = epoch
    summary = dict([("epoch", epoch)] + list(summary.items()))
    df = pd.DataFrame([summary])
    write_header = not os.path.exists(csv_path)
    df.to_csv(csv_path, mode='a', header=write_header, index=False)
    #print(f"\nSaved metrics to: {csv_path}")
    return mAP50

import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

def show_predictions(model, dataset, device, score_threshold=0.25):
    model.eval()
    output_dir="results/predictions"
    os.makedirs(output_dir, exist_ok=True)

    for idx in range(len(dataset)):
        img, target = dataset[idx]
        img_tensor = img.to(device).unsqueeze(0)

        with torch.no_grad():
            output = model(img_tensor)[0]

        # Filtro predizioni con punteggio basso
        keep = output['scores'] >= score_threshold
        pred_boxes = output['boxes'][keep].cpu()
        pred_labels = output['labels'][keep].cpu()
        scores = output['scores'][keep].cpu()

        # Disegna box predetti
        pred_img = draw_bounding_boxes((img * 255).byte(), pred_boxes, labels=[f"P:{l.item()}" for l in pred_labels],
                                       colors="red", width=1)

        # Disegna box reali
        # true_boxes = target['boxes'].cpu()
        # true_labels = target['labels'].cpu()
        # full_img = draw_bounding_boxes(pred_img, true_boxes, labels=[f"T:{l.item()}" for l in true_labels],
        #                                colors="green", width=2)

        

        # Nome originale immagine
        original_filename = dataset.image_files[idx] # Es: "img001.jpg"
        output_filename = os.path.splitext(original_filename)[0] + "_pred.jpg"

        # Salva immagine
        img_pil = to_pil_image(pred_img)
        img_pil.save(os.path.join(output_dir, output_filename))
        
    print(f"saved prediction to: {output_dir}")



def main():
    gc.collect()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    dataset_root = "dataset"
    save_weights_dir = "results/weights"
    num_epochs = 5
    writer = SummaryWriter("logs/faster_rcnn50")

    transforms = T.Compose([
        T.ToTensor()
    ])

    dataset_train = VOCDataset(dataset_root, image_set="train", transforms=transforms)
    dataset_valid = VOCDataset(dataset_root, image_set="val", transforms=transforms)
    dataset_test = VOCDataset(dataset_root, image_set="test", transforms=transforms)
    data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
    data_loader_valid = DataLoader(dataset_valid, batch_size=2, shuffle=True, collate_fn=collate_fn)
    #data_loader_test = DataLoader(dataset_test, batch_size=2, shuffle=True, collate_fn=collate_fn)

    model = get_model(num_classes=len(dataset_train.classes))
    model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    best_score_valid = 0.0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        total_loss_valid = 0.0

        for images, targets in tqdm(data_loader_train, desc=f"Epoch {epoch+1}"):
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            total_loss += losses.item()
        print("Loss sul test set:")
        for images, targets in tqdm(data_loader_valid, desc=f"Epoch {epoch+1}"):
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            total_loss_valid += losses.item()

            
        writer.add_scalar("loss/train", total_loss, epoch+1)
        writer.add_scalar("loss/valid", total_loss_valid, epoch+1)
        lr_scheduler.step()
        print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

        #EVALUATION PER EPOCA SUL TRAIN SET
        # print("Score sul train set:")
        # evaluate_metrics(model, data_loader_train, device, epoch=epoch+1,csv_path="results/train_metrics.csv")

        #EVALUATION PER EPOCA SUL VALID SET
        print("Score sul valid set:")
        current_score_valid = evaluate_metrics(model, data_loader_valid, device, epoch=epoch+1,csv_path="results/valid_metrics.csv")

        print("\n")

        os.makedirs(save_weights_dir, exist_ok=True)
        torch.cuda.empty_cache()
        if current_score_valid > best_score_valid:
            best_score_valid = current_score_valid
            torch.save(model.state_dict(), "results/weights/fasterrcnn_voc_best.pth")
            #print(f"New best model saved with score: {best_score:.4f}")

        torch.save(model.state_dict(), "results/weights/fasterrcnn_voc_last.pth")

    print("Training complete. Model saved.")
    #fase di inferenza
    
    show_predictions(model,dataset_test,device=device,score_threshold=0.25)
    torch.cuda.empty_cache()
    

if __name__ == "__main__":
    main()


#mettere nel file csv le metriche di ogni epoca ✅
#inserire weight and biases (oppure tensorboard) ✅
#salvare il best.pth del modello ✅
#fare la evaluation sul valid set oltre a quella di train(aggiungi la loss/val su tensorboard), salvare il best.pth in base alla map sul valid set(segna epoca nel file) ✅
#refactor con divisione in celle, oppure vari file.py
#dare le etichette nelle classi 



c:\Users\Edoardo\anaconda3\envs\yolo\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Edoardo\anaconda3\envs\yolo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1: 100%|██████████| 65/65 [00:07<00:00,  8.76it/s]


Loss sul test set:


Epoch 1: 100%|██████████| 13/13 [00:06<00:00,  2.09it/s]


Epoch 1 Loss: 33.0811
Score sul valid set:


Evaluating: 100%|██████████| 13/13 [00:05<00:00,  2.35it/s]


...

Epoch 5 Loss: 10.4907
Score sul valid set:


Evaluating: 100%|██████████| 13/13 [00:05<00:00,  2.49it/s]


Evaluation Metrics:
map_50: 0.8467138409614563


Training complete. Model saved.


c:\Users\Edoardo\anaconda3\envs\yolo\Lib\site-packages\torchvision\utils.py:211: UserWarning: boxes doesn't contain any box. No box was drawn
  warnings.warn("boxes doesn't contain any box. No box was drawn")


saved prediction to: results/predictions
